In [88]:
# !pip install opencv-python

LOADING FUCTIONS , LIBRARIES AND DEPENDENCIES

In [12]:
from keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import cv2
import os
from imutils.video import VideoStream
import imutils
import matplotlib.pyplot as plt

PROTOTXTPATH = SKELEON OF DEEP NEURAL NETWORK MODEL IMPORTED FORM CV2'S REPOSITRIES  OF GITHUB 
WEIGHTSPATH = WEIGHTS OF THE DNN MODEL FOR FACE RECOGNITION DOWNLOADED FROM GITHUB'S REPOSITRIES OF CV2 

In [13]:
# from os.path import dirname, join,__file__
prototxtPath = 'deploy.prototxt.txt'
weightsPath= 'res10_300x300_ssd_iter_140000.caffemodel'

DNN MODEL FOR FACE RECOGNITION IS LOADED IN faceNet

In [14]:
faceNet=cv2.dnn.readNet(weightsPath,prototxtPath)

In [15]:
type(faceNet)

cv2.dnn.Net

MODEL WHICH WAS SAVED FOR MASK DETECTION IS LOADED IN model

In [16]:
## Load model
model=load_model('model/custom_4000_32_100.h5')

RGB IMAGE IS CONVERTED TO HSV IMAGE 
THEN H S V COMPONENTS ARE SPLITTED (HUE,SATURATION,VALUE) 
AND THEN V COMPONENT IS INCREASED BY 30 
THEN IMAGE IS MERGED TOGETHER AND THEN CONVERTED TO ITS ORIGNAL FORM RGB 

In [17]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

 1. FRAME PASSED IS CONVERTED TO BLOB 
 2. THEN IT IS PASSED IN FACE DETCTION MODEL 
 3. IF IT SUCESSFULLY DETECTS IT AS A FACE THEN IT FIRSTLY INHANCES THE PROPERTIES OF THE IMAGE AND THEN THE LOCATION (CO-ORDINATES) IS ADDED TO THE LIST  
 4. THEN FOR EVERY FACE IT PREDICTS THE FACE IS MASKED OR NOT AND THEN LOCATION OF FACE AND IT'S PREDICTION IS RETURNED 

In [18]:
def detect_and_predict_mask(frame,faceNet,model):
    #grab the dimensions of the frame and then construct a blob
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    #initialize our list of faces, their corresponding locations and list of predictions
    
    faces=[]
    locs=[]
    preds=[]
    
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
    
    
        if confidence>0.5: #if it is a face 
        #we need the X,Y coordinates
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])    #WIDTH AND HEIGHT
            (startX,startY,endX,endY)=box.astype('int')      #STORED IN BOX 
        
            #ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY)=(max(0,startX-30),max(0,startY-30))
            (endX,endY)=(min(w-1,endX+30), min(h-1,endY+30))
        
            #extract the face ROI, convert it from BGR to RGB channel, resize it to 96,96 and preprocess it
            face=frame[startY:endY, startX:endX]
            face=increase_brightness(face)
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(96,96))
            face=img_to_array(face)
            face=preprocess_input(face)
        
            faces.append(face)
            locs.append((startX,startY,endX,endY))
        
    #only make a predictions if atleast one face was detected
    if len(faces)>0:
        faces=np.array(faces,dtype='float32')
        preds=model.predict(faces,batch_size=6)
        
    return (locs,preds)

VIDEO IS USED 
FRAME IS READ AND THEN RESIZED 
LOCATIONS OF FACES AND ITS PRECTIONS ARE THEN SAVED
THEN FOR EVERY BOX IN THAT FRAME THE WE SHOW ITS LABEL IN REAL TIME WITH PERCENTAGE OF CONFIDENCE  

In [19]:
vs=VideoStream(src=0).start()

while True:
    #grab the frame from the threaded video stream and resize it
    #to have a maximum width of 400 pixels
    frame=vs.read()
    frame=imutils.resize(frame,width=800)
    #detect faces in the frame and preict if they are waring masks or not
    (locs,preds)=detect_and_predict_mask(frame,faceNet,model)
    
    #loop over the detected face locations and their corrosponding loactions
    
    for (box,pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>0.5 else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        #display the label and bounding boxes
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
        
    #show the output frame
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
    
    if key==ord('q'):
        break
        
vs.stop()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step
